# Imports

In [ ]:
!pip install gspread
!pip install oauth2client
!pip install slackclient
!pip install -U snowflake-connector-python
!pip install -U snowflake-snowpark-python
!pip install --upgrade psycopg2
!pip install -U sqlalchemy
!pip install df2gspread

In [3]:
from snowflake.snowpark import Session 
import os
import boto3
import base64
import json
from requests import get
from pathlib import Path
from datetime import datetime, timedelta, date
import time
import zipfile
import io
import sqlalchemy
from pathlib import Path
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint
import pandas as pd
import numpy as np
import psycopg2
import sys
import requests
from botocore.exceptions import ClientError
import importlib
from io import StringIO
import snowflake.connector
from sqlalchemy import create_engine
import asyncio
import nest_asyncio
import re
import pytz
import pandas as pd
import snowflake.connector

# Defining Functions

In [4]:
import os
import boto3
import base64
from botocore.exceptions import ClientError
import json
from requests import get
from pathlib import Path
from datetime import datetime

def imports():
        import os
        
        import pandas as pd
        import numpy as np
        import psycopg2

import json
import boto3
import os
from pathlib import Path

def get_secret(secret_name):
    region_name = "us-east-1"

    # Create a Secrets Manager client
    session = boto3.session.Session()
    client = session.client(
        service_name='secretsmanager',
        region_name=region_name
    )

    # In this sample we only handle the specific exceptions for the 'GetSecretValue' API.
    # See https://docs.aws.amazon.com/secretsmanager/latest/apireference/API_GetSecretValue.html
    # We rethrow the exception by default.

    try:
        get_secret_value_response = client.get_secret_value(
            SecretId=secret_name
        )
    except ClientError as e:
        if e.response['Error']['Code'] == 'DecryptionFailureException':
            # Secrets Manager can't decrypt the protected secret text using the provided KMS key.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InternalServiceErrorException':
            # An error occurred on the server side.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidParameterException':
            # You provided an invalid value for a parameter.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'InvalidRequestException':
            # You provided a parameter value that is not valid for the current state of the resource.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
        elif e.response['Error']['Code'] == 'ResourceNotFoundException':
            # We can't find the resource that you asked for.
            # Deal with the exception here, and/or rethrow at your discretion.
            raise e
    else:
        # Decrypts secret using the associated KMS CMK.
        # Depending on whether the secret is a string or binary, one of these fields will be populated.
        if 'SecretString' in get_secret_value_response:
            return get_secret_value_response['SecretString']
        else:
            return base64.b64decode(get_secret_value_response['SecretBinary'])
        
def initialize_env():
    snowflake_sg_secret = json.loads(get_secret("Snowflake-sagemaker"))
    dwh_writer_secret = json.loads(get_secret("prod/db/datawarehouse/sagemaker"))
    metabase_secret = json.loads(get_secret("prod/metabase/maxab_config"))

    os.environ["SNOWFLAKE_USERNAME"] = snowflake_sg_secret["username"]
    os.environ["SNOWFLAKE_PASSWORD"] = snowflake_sg_secret["password"]
    os.environ["SNOWFLAKE_ACCOUNT"] = snowflake_sg_secret["account"]
    os.environ["SNOWFLAKE_DATABASE"] = snowflake_sg_secret["morocco_database"]

    os.environ["DWH_WRITER_HOST_NEW"] = dwh_writer_secret["host"]
    os.environ["DWH_WRITER_NAME_NEW"] = dwh_writer_secret["dbname"]
    os.environ["DWH_WRITER_USER_NAME_NEW"] = dwh_writer_secret["username"]
    os.environ["DWH_WRITER_PASSWORD_NEW"] = dwh_writer_secret["password"]
    
    os.environ["AFRICA_METABASE_USERNAME"] = metabase_secret["metabase_user"]
    os.environ["AFRICA_METABASE_PASSWORD"] = metabase_secret["metabase_password"]
    os.environ["AFRICA_METABASE_URL"] = metabase_secret["metabase_morocco_site"]

    json_path_sheets = str(Path.home()) + "/service_account_key_sheets.json"
    sheets_key = get_secret("prod/maxab-sheets")
    f = open(json_path_sheets, "w")
    f.write(sheets_key)
    f.close()
    os.environ["GOOGLE_APPLICATION_CREDENTIALS_SHEETS"] = json_path_sheets
    

def get_from_gsheet(workbook, sheet):
    scope = ["https://spreadsheets.google.com/feeds",
         'https://www.googleapis.com/auth/spreadsheets',
         "https://www.googleapis.com/auth/drive.file",
         "https://www.googleapis.com/auth/drive"]
    initialize_env()
    creds = ServiceAccountCredentials.from_json_keyfile_name(os.environ["GOOGLE_APPLICATION_CREDENTIALS_SHEETS"], scope)
    client = gspread.authorize(creds)
    try:
        wks = client.open(workbook).worksheet(sheet)
        sheet = pd.DataFrame(wks.get_all_records())
    except:
        print(sheet,'failed to fetch data')
    
    return sheet

In [5]:
import oauth2client
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pprint import pprint
from df2gspread import df2gspread as d2g
import pandas as pd
import os
import numpy as np
import psycopg2
import sys
import requests

import importlib

import os
import psycopg2
import numpy as np

from datetime import datetime, timedelta, date
import datetime
import copy
import pandas as pd
from tqdm import tqdm
import time
import datetime as dt

initialize_env()
from oauth2client.service_account import ServiceAccountCredentials
# define the scope
scope = ['https://spreadsheets.google.com/feeds','https://www.googleapis.com/auth/drive']
# add credentials to the account
creds = ServiceAccountCredentials.from_json_keyfile_name(os.environ["GOOGLE_APPLICATION_CREDENTIALS_SHEETS"], scope)
# authorize the clientsheet
client = gspread.authorize(creds)

ModuleNotFoundError: No module named 'df2gspread'

In [5]:
import os
import requests
import json
import pandas as pd
import numpy as np
import re
import datetime
import seaborn as sns
from io import StringIO, BytesIO
import sys
from datetime import datetime

def ret_metabase(base_url, username, password, card_id):
    card_id = str(card_id)

    initialize_env()

    base_url = base_url + 'api'
    base_headers = {'Content-Type': 'application/json'}

    try:
        s_response = requests.post(
            base_url + '/session',
            data=json.dumps({
                'username': username,
                'password': password
            }),
            headers=base_headers)

        s_response.raise_for_status()

        session_token = s_response.json()['id']
        base_headers['X-Metabase-Session'] = session_token

        p_response = requests.post(
            base_url + '/card/' + card_id + '/query/xlsx',
            headers=base_headers
        )
        p_response.raise_for_status()

        my_dict = p_response.content
        xlsx_data = BytesIO(my_dict)
        df = pd.read_excel(xlsx_data, engine='openpyxl')
        df.columns = df.columns.str.lower()
        return df
    except Exception as e:
        raise e
    
#run query save on metabse with ID, 
#for example here query 1606 is save in my personal collection and I can run it like line below
# ret_metabase( os.environ["AFRICA_METABASE_URL"],os.environ["AFRICA_METABASE_USERNAME"],os.environ["AFRICA_METABASE_PASSWORD"],9707)

Matplotlib is building the font cache; this may take a moment.


# Functions

In [6]:
## assigning data in any project
def assign_data(df, list):

    actual_length = len(df) % len(list)
    rows_needed = len(list) - actual_length
    columns_needed = len(df.axes[1])
    print(rows_needed)
    print(columns_needed)
    df = df.sample(frac=1)
    empty_df = pd.DataFrame(index=np.arange(rows_needed), columns=np.arange(columns_needed))
    assgn = pd.concat([df, empty_df])
    assgn = assgn.dropna(axis=1, how='all')
    mplr = len(assgn) / len(list) 
    assgn = assgn.assign(agent_assigned=[val for val in list for _ in range(int(mplr))])
    assgn = assgn.dropna()
    # print(assgn)
    if assgn.shape[1] != 1:

            assgn["retailer_id"] = assgn["retailer_id"].fillna('').astype(str).str.replace(".0","",regex=False)
            assgn["mobile"] = assgn["mobile"].fillna('').astype(str).str.replace(".0","",regex=False)
            # df = assgn.groupby('agent_assigned').head(assigns)
            return assgn
    else:
            assgn.drop(['agent_assigned'], axis=1)
            assgn = assgn.iloc[0:0]
            assgn = assgn.assign(index="", retailer_id = "",name = "", mobile="",sales_order_id="",created_at="",project_name="",description="",agent_assigned="")
            assgn = assgn[["index","retailer_id","name","mobile","sales_order_id","created_at","project_name","description","agent_assigned"]]
            # df = assgn.groupby('agent_assigned').head(assigns)
            return assgn

In [1]:
!pip install gspread_dataframe


In [2]:
import pytz
from datetime import datetime

initialize_env()

# Time zone for Morocco
morocco_tz = pytz.timezone('Africa/Casablanca')

# Get current time in Moroccan time zone
now = datetime.now(morocco_tz)
current_time = now.time()
current_day = now.strftime('%A')
print(current_day)
hour = int(str(now.time())[0:2])
# print(now)
# print(current_day)
# print(hour)

#Agents sheet
scope = [
    "https://spreadsheets.google.com/feeds",
    'https://www.googleapis.com/auth/spreadsheets',
    "https://www.googleapis.com/auth/drive.file",
    "https://www.googleapis.com/auth/drive"
]

creds = ServiceAccountCredentials.from_json_keyfile_name(os.environ["GOOGLE_APPLICATION_CREDENTIALS_SHEETS"], scope)
client = gspread.authorize(creds)

sheet = client.open('Fintech Runsheet')
agents_sheet_instance = sheet.worksheet('agents_sheet')
agents_df = pd.DataFrame(agents_sheet_instance.get())
agents_df.columns = agents_df.iloc[0]
agents_df = agents_df[1:]
agents_df.reset_index(drop=True, inplace=True)
print(agents_df)
# agents_df
print(hour)
#Agents Assignment 
agents_df_copy = agents_df.copy()
agents_df_copy['assignment_start_time'] = agents_df['start_time'].astype(int)
agents_df_copy['assignment_end_time'] = agents_df['end_time'].astype(int)
agents_df_copy['agents_days_off'] = agents_df['days_off'].str.split(', ')

agents_df_copy['assign_data'] =  np.where((hour >= agents_df_copy['assignment_start_time'] )
                                &
                                (hour <= agents_df_copy['assignment_end_time'])
                                & 
                                (str(current_day) not in str(agents_df_copy['agents_days_off']) )
                                , 'yes','no' )

# agents_df_copy['assign_data'] =  np.where(agents_df['agent_id']==108
#                                 , 'yes','no' )

print(agents_df_copy)
available_agents = agents_df_copy.loc[agents_df_copy['assign_data'] =='yes']
available_agents_list = available_agents['agent_id'].values.tolist()
number_of_available_agents = len(available_agents_list)
# print(number_of_available_agents)


if number_of_available_agents != 0:
    
    #en route query 
    df_1 = ret_metabase( os.environ["AFRICA_METABASE_URL"],os.environ["AFRICA_METABASE_USERNAME"],os.environ["AFRICA_METABASE_PASSWORD"],10000)
    df_1.columns = map(str.lower, df_1.columns)
    df_2 = ret_metabase( os.environ["AFRICA_METABASE_URL"],os.environ["AFRICA_METABASE_USERNAME"],os.environ["AFRICA_METABASE_PASSWORD"],10085)
    df_2.columns = map(str.lower, df_2.columns)
    df_3 = ret_metabase( os.environ["AFRICA_METABASE_URL"],os.environ["AFRICA_METABASE_USERNAME"],os.environ["AFRICA_METABASE_PASSWORD"],10368)
    df_3.columns = map(str.lower, df_3.columns)
    df_4 = ret_metabase( os.environ["AFRICA_METABASE_URL"],os.environ["AFRICA_METABASE_USERNAME"],os.environ["AFRICA_METABASE_PASSWORD"],10410)
    df_4.columns = map(str.lower, df_4.columns)

    df_5 = ret_metabase( os.environ["AFRICA_METABASE_URL"],os.environ["AFRICA_METABASE_USERNAME"],os.environ["AFRICA_METABASE_PASSWORD"],10728)
    df_5.columns = map(str.lower, df_5.columns)
    
    df_6 = ret_metabase( os.environ["AFRICA_METABASE_URL"],os.environ["AFRICA_METABASE_USERNAME"],os.environ["AFRICA_METABASE_PASSWORD"],14451)
    df_6.columns = map(str.lower, df_6.columns)
    
    
    # Concatenate the dataframes along the rows (union all)
    dataframes = [df_1,df_2,df_3,df_5,df_6]
    df_unfiltered = pd.concat(dataframes, ignore_index=True)
    df_filtered = df_unfiltered.drop_duplicates(subset=['retailer_id'], keep="first")
    df_filtered = df_filtered.reset_index(drop=True)
    print(df_filtered)
    
    #Remove the data that already exists if found 
    sheet = client.open('Fintech Runsheet')
    data_instance = sheet.worksheet('Data')
    data_instance_df= pd.DataFrame(data_instance.get('F5:F'))
    print('data_instant_df')
    print(data_instance_df)

    if(data_instance_df.empty==False):
        data_instance_df.columns = ['retailer_id']
        main_data_to_assign = pd.DataFrame(df_filtered.loc[~df_filtered['retailer_id'].isin(data_instance_df['retailer_id'].astype(int).values)])
        main_data_assigned = assign_data(main_data_to_assign,available_agents_list)
        # print(main_data_assigned)
    else:
        main_data_to_assign = df_filtered.copy()
        main_data_assigned = assign_data(main_data_to_assign,available_agents_list)
        # print(main_data_assigned)
        
    # agents_df.columns = agents_df.iloc[0]
    # agents_df = agents_df[1:]
    # agents_df.reset_index(drop=True, inplace=True)
    # agents_df
#     grouped_counts = main_data_assigned.groupby('agent_assigned').size().reset_index(name='count')
#     print(grouped_counts)

    #importing data in agents' sheet
    final_data_to_assign = main_data_assigned.copy()
    # print(final_data_to_assigny)
    # final_data_to_assign = final_data_to_assign.drop_duplicates(subset='retailer_id', keep="first")
    final_data_to_assign['added_at'] = now.strftime('%Y-%m-%d %H:%M')

    if final_data_to_assign.columns[0] == 'index':
        final_data_to_assign = final_data_to_assign.drop(labels='index', axis=1)
    pass

    final_data_to_assign = final_data_to_assign.astype(str)
    google_sh = client.open('Fintech Runsheet')
    sheet = google_sh.worksheet('raw_data')
    # sheet.clear()
    sheet.append_rows([final_data_to_assign.columns.values.tolist()] + final_data_to_assign.values.tolist(), value_input_option="USER_ENTERED")
else:
    print("no agents available")


NameError: name 'initialize_env' is not defined

In [13]:
#Agents sheet
scope = [
    "https://spreadsheets.google.com/feeds",
    'https://www.googleapis.com/auth/spreadsheets',
    "https://www.googleapis.com/auth/drive.file",
    "https://www.googleapis.com/auth/drive"
]

creds = ServiceAccountCredentials.from_json_keyfile_name(os.environ["GOOGLE_APPLICATION_CREDENTIALS_SHEETS"], scope)
client = gspread.authorize(creds)

sheet = client.open('[HOURLY] Fintech Runsheet')
data_instance = sheet.worksheet('Data')
data_instance_df= pd.DataFrame(data_instance.get('E5:E'),columns=['sales_order_id'])
print(data_instance_df)

SpreadsheetNotFound: <Response [200]>

In [ ]:
df_1 = ret_metabase( os.environ["AFRICA_METABASE_URL"],os.environ["AFRICA_METABASE_USERNAME"],os.environ["AFRICA_METABASE_PASSWORD"],10050)
df_1.columns = map(str.lower, df_1.columns)
df_1 = df_1.drop_duplicates(subset=['retailer_id'], keep="first")
df_1 = df_1.reset_index(drop=True)
print(df_1)

available_agents_list = [109,108]

#Remove the data that already exists if found 
sheet = client.open('[HOURLY] Fintech Runsheet')
data_instance = sheet.worksheet('Data')
data_instance_df= pd.DataFrame(data_instance.get('E5:E'))
print(data_instance_df)

if(data_instance_df.empty==False):
    data_instance_df.columns = ['sales_order_id']
    main_data_to_assign = pd.DataFrame(df_1.loc[~df_1['sales_order_id'].isin(data_instance_df['sales_order_id'].astype(int).values)])
    main_data_assigned = assign_data(main_data_to_assign,available_agents_list)
    print(main_data_assigned)
else:
    main_data_to_assign = df_1.copy()
    main_data_assigned = assign_data(main_data_to_assign,available_agents_list)
    print(main_data_assigned)



#importing data in agents' sheet
final_data_to_assign = main_data_assigned.copy()
final_data_to_assign = final_data_to_assign
# final_data_to_assign = final_data_to_assign.drop_duplicates(subset='retailer_id', keep="first")
final_data_to_assign['added_at'] = now.strftime('%Y-%m-%d %H:%M')

if final_data_to_assign.columns[0] == 'index':
    final_data_to_assign = final_data_to_assign.drop(labels='index', axis=1)
pass

final_data_to_assign = final_data_to_assign.astype(str)
google_sh = client.open('[HOURLY] Fintech Runsheet')
sheet = google_sh.worksheet('raw_data')
# sheet.clear()
sheet.append_rows([final_data_to_assign.columns.values.tolist()] + final_data_to_assign.values.tolist(), value_input_option="USER_ENTERED")


